In [29]:
import pymysql.cursors
database = 'healthCC'

def create_connection(db_file):
    """ create a database connection to the MySQL database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        connection = pymysql.connect(host='localhost',
                             user='root',
                             password='root',
                             db=db_file)
        return connection
    except pymysql.InternalError as e:
        return False

conn = create_connection(database)
if conn is not False:
    cur = conn.cursor(pymysql.cursors.DictCursor)
    sql = "SELECT name FROM hospitals"
    cur.execute(sql)
    for row in cur:
        print(row['name'])
    ##fetch data
#db1 = pymysql.connect(host="localhost",user="root",passwd="root")
#cursor = connection.cursor()
#sql = 'CREATE DATABASE mydata'
#cursor.execute(sql)

#API_KEY = '6L9YC3lPsRpqnJ9q_YI1_aq046LPIpBRb0ZKqIQSX8bm48Ke1KHk6J6DYMKJFDNLD-ycIWSkC5EPNTY4AVTqPjlrGHYzEM2bCjpQ-9L5hoNWxf02x_zgfoZzWIeRWnYx'

# -*- coding: utf-8 -*-
"""
Yelp Fusion API code sample.

This program demonstrates the capability of the Yelp Fusion API
by using the Search API to query for businesses by a search term and location,
and the Business API to query additional information about the top result
from the search query.

Please refer to http://www.yelp.com/developers/v3/documentation for the API
documentation.

This program requires the Python requests library, which you can install via:
`pip install -r requirements.txt`.

Sample usage of the program:
`python sample.py --term="bars" --location="San Francisco, CA"`
"""
from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib


# This client code can run on Python 2.x or 3.x.  Your imports can be
# simpler if you only need one of those.
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode


# Yelp Fusion no longer uses OAuth as of December 7, 2017.
# You no longer need to provide Client ID to fetch Data
# It now uses private keys to authenticate requests (API Key)
# You can find it on
# https://www.yelp.com/developers/v3/manage_app
API_KEY = '6L9YC3lPsRpqnJ9q_YI1_aq046LPIpBRb0ZKqIQSX8bm48Ke1KHk6J6DYMKJFDNLD-ycIWSkC5EPNTY4AVTqPjlrGHYzEM2bCjpQ-9L5hoNWxf02x_zgfoZzWIeRWnYx'


# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.


# Defaults for our simple example.
DEFAULT_TERM = 'dinner'
DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 10


def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.

    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.

    Returns:
        dict: The JSON response from the request.

    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(api_key, term, location, categories):
    """Query the Search API by a search term and location.

    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.

    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'cateogires': categories,
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)


def get_business(api_key, business_id):
    """Query the Business API by a business ID.

    Args:
        business_id (str): The ID of the business to query.

    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, api_key)

def parseJson(jsonData):
    #pprint.pprint(jsonData)
    yelp_id = jsonData['id']
    rating = jsonData['rating']
    review_count = jsonData['review_count']
    url = jsonData['url'].split('?')
    latitude = jsonData['coordinates']['latitude']
    longitude = jsonData['coordinates']['longitude']
    
    print(yelp_id, rating, review_count, url[0], latitude, longitude)
    
def query_api(term, location, categories):
    """Queries the API by the input values from the user.

    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(API_KEY, term, location, categories)

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return

    business_id = businesses[0]['id']

    #print(u'{0} businesses found, querying business info ' \
    #    'for the top result "{1}" ...'.format(
    #        len(businesses), business_id))
    response = get_business(API_KEY, business_id)

    #print(u'Result for business "{0}" found:'.format(business_id))
    #pprint.pprint(response, indent=2)
    parseJson(response)


def main():
    
    term = "CATHOLIC MEDICAL CENTER"
    # City, State, Zip
    location = "MANCHESTER NH 3102"
    categories = "MEDICAL CENTERS"
    try:
        query_api(term, location, categories)
    except HTTPError as error:
        print(error.code, error.read())


if __name__ == '__main__':
    main()

SOUTHEAST ALABAMA MEDICAL CENTER
MARSHALL MEDICAL CENTER SOUTH
ELIZA COFFEE MEMORIAL HOSPITAL
MIZELL MEMORIAL HOSPITAL
CRENSHAW COMMUNITY HOSPITAL
ST VINCENT'S EAST
DEKALB REGIONAL MEDICAL CENTER
SHELBY BAPTIST MEDICAL CENTER
CALLAHAN EYE HOSPITAL
HELEN KELLER MEMORIAL HOSPITAL
DALE MEDICAL CENTER
CHEROKEE MEDICAL CENTER
BAPTIST MEDICAL CENTER SOUTH
JACKSON HOSPITAL & CLINIC INC
EAST ALABAMA MEDICAL CENTER
WEDOWEE HOSPITAL
UNIVERSITY OF ALABAMA HOSPITAL
COMMUNITY HOSPITAL INC
CULLMAN REGIONAL MEDICAL CENTER
ANDALUSIA REGIONAL HOSPITAL
STRINGFELLOW MEMORIAL HOSPITAL
HUNTSVILLE HOSPITAL
GADSDEN REGIONAL MEDICAL CENTER
MARION REGIONAL MEDICAL CENTER
FAYETTE MEDICAL CENTER
RIVERVIEW REGIONAL MEDICAL CENTER
GEORGIANA MEDICAL CENTER
MEDICAL CENTER ENTERPRISE
GREENE COUNTY HOSPITAL
LAKE MARTIN COMMUNITY HOSPITAL
FLOWERS HOSPITAL
ST VINCENT'S BIRMINGHAM
BIBB MEDICAL CENTER
LAWRENCE MEDICAL CENTER
HIGHLANDS MEDICAL CENTER
WIREGRASS MEDICAL CENTER
RUSSELL HOSPITAL
MEDICAL CENTER BARBOUR
CLAY COU

MILLER CHILDREN'S HOSPITAL
SHRINERS HOSPITALS FOR CHILDREN
SHRINERS HOSPITALS FOR CHILDREN NORTHERN CALIF
NORTH COLORADO MEDICAL CENTER
LONGMONT UNITED HOSPITAL
PLATTE VALLEY MEDICAL CENTER
MONTROSE MEMORIAL HOSPITAL
SAN LUIS VALLEY HEALTH
LUTHERAN MEDICAL CENTER
POUDRE VALLEY HOSPITAL
DENVER HEALTH MEDICAL CENTER
CENTURA HEALTH-ST MARY CORWIN MEDICAL CENTER
MERCY REGIONAL MEDICAL CENTER
PRESBYTERIAN ST LUKES MEDICAL CENTER
CENTURA HEALTH-ST ANTHONY HOSPITAL
CENTURA HEALTH-ST THOMAS MORE HOSPITAL
PARKVIEW MEDICAL CENTER INC
UNIVERSITY COLO HEALTH MEMORIAL HOSPITAL CENTRAL
ST MARYS  MEDICAL CENTER
UNIVERSITY OF COLORADO HOSPITAL AUTHORITY
FOOTHILLS HOSPITAL
SAINT JOSEPH HOSPITAL
MCKEE MEDICAL CENTER
CENTURA HEALTH-PENROSE ST FRANCIS HEALTH SERVICES
ROSE MEDICAL CENTER
SWEDISH MEDICAL CENTER
KEEFE MEMORIAL HOSPITAL
COLORADO PLAINS MEDICAL CENTER
YAMPA VALLEY MEDICAL CENTER
COMMUNITY HOSPITAL
CENTURA HEALTH-PORTER ADVENTIST HOSPITAL
NORTH SUBURBAN MEDICAL CENTER
DELTA COUNTY MEMORIAL HOSP

MT SINAI HOSPITAL MEDICAL CENTER
HSHS GOOD SHEPHERD HOSPITAL INC
COPLEY MEMORIAL HOSPITAL
SHERMAN HOSPITAL
ST ANTHONYS MEMORIAL HOSPITAL
VISTA MEDICAL CENTER WEST
SSM HEALTH ST MARY'S HOSPITAL -CENTRALIA
GALESBURG COTTAGE HOSPITAL
CGH MEDICAL CENTER
GOOD SAMARITAN REGIONAL HLTH CENTER
ADVOCATE TRINITY HOSPITAL
WEST SUBURBAN MEDICAL CENTER
OSF SAINT ANTHONY'S HEALTH CENTER
ST JOHNS HOSPITAL
MACNEAL  HOSPITAL
PASSAVANT AREA HOSPITAL
JERSEY COMMUNITY HOSPITAL
PALOS COMMUNITY HOSPITAL
RUSH OAK PARK HOSPITAL
ST MARY MEDICAL CENTER
ADVENTIST LA GRANGE MEMORIAL HOSPITAL
SAINT FRANCIS MEDICAL CENTER
ROSELAND COMMUNITY HOSPITAL
TOUCHETTE REGIONAL HOSPITAL INC
PRESENCE SAINT FRANCIS HOSPITAL
LOUIS A WEISS MEMORIAL HOSPITAL
LORETTO HOSPITAL
VISTA MEDICAL CENTER EAST
THE UNIVERSITY OF CHICAGO MEDICAL CENTER
MC DONOUGH DISTRICT HOSPITAL
THE CARLE FOUNDATION HOSPITAL
PRESENCE UNITED SAMARITANS MEDICAL CENTER
SAINT ANTHONY HOSPITAL
MIDWESTERN REGION MED CENTER
MORRIS HOSPITAL & HEALTHCARE CENTERS
ST 

GRUNDY COUNTY MEMORIAL HOSPITAL
CHI HEALTH  - MERCY CORNING
POCAHONTAS COMMUNITY HOSPITAL
JONES REGIONAL MEDICAL CENTER
HANCOCK COUNTY HEALTH SYSTEM
FRANKLIN GENERAL HOSPITAL
CHI HEALTH MISSOURI VALLEY
ADAIR COUNTY MEMORIAL HOSPITAL
HAWARDEN REGIONAL HEALTHCARE
GUTTENBERG MUNICIPAL HOSPITAL
GENESIS MEDICAL CENTER-DEWITT
GUTHRIE COUNTY HOSPITAL
KEOKUK COUNTY HEALTH CENTER
GUNDERSEN PALMER LUTHERAN HOSPITAL AND CLINICS
MARENGO MEMORIAL HOSPITAL
VETERANS MEMORIAL HOSPITAL
CENTRAL COMMUNITY HOSPITAL
COMMUNITY MEMORIAL HOSPITAL MEDICAL CENTER
SANFORD ROCK RAPIDS MEDICAL CENTER
DALLAS COUNTY HOSPITAL
MITCHELL COUNTY REGIONAL HEALTH
GEORGE C GRAPE COMMUNITY HOSPITAL
GREENE COUNTY MEDICAL CENTER
MADISON COUNTY MEMORIAL HOSPITAL
DAVIS COUNTY HOSPITAL
REGIONAL HEALTH SERVICES OF HOWARD COUNTY
JACKSON COUNTY REGIONAL HEALTH CENTER
AUDUBON COUNTY MEMORIAL HOSPITAL
MERCY MEDICAL CENTER-NEW HAMPTON
MANNING REGIONAL HEALTHCARE CENTER
STORY COUNTY HOSPITAL
HUMBOLDT COUNTY MEMORIAL HOSPITAL
BUCHANAN CO

ADVENTIST HEALTHCARE SHADY GROVE MEDICAL CENTER
UNIV OF MD REHABILITATION &  ORTHOPAEDIC INSTITUTE
FORT WASHINGTON HOSPITAL
ATLANTIC GENERAL HOSPITAL
MEDSTAR SOUTHERN MARYLAND HOSPITAL CENTER
UNIVERSITY OF MARYLAND ST JOSEPH MEDICAL CENTER
LEVINDALE HEBREW GERIATRIC CENTER AND HOSPITAL
HOLY CROSS GERMANTOWN HOSPITAL
MOUNT WASHINGTON PEDIATRIC HOSPITAL
KENNEDY KRIEGER INSTITUTE
HEALTHALLIANCE HOSPITALS, INC
MOUNT AUBURN HOSPITAL
STURDY MEMORIAL HOSPITAL
LAWRENCE GENERAL HOSPITAL
CAMBRIDGE HEALTH ALLIANCE
CAPE COD HOSPITAL
COOLEY DICKINSON HOSPITAL INC,THE
BAYSTATE FRANKLIN MEDICAL CENTER
CARNEY HOSPITAL
HARRINGTON MEMORIAL HOSPITAL-1
SAINT ANNE'S HOSPITAL
HOLYOKE MEDICAL CENTER
ANNA JAQUES HOSPITAL
BAYSTATE WING HOSPITAL AND MEDICAL CENTERS
BOSTON MEDICAL CENTER CORPORATION-
BEVERLY HOSPITAL CORPORATION
NORTH SHORE MEDICAL CENTER -
ST ELIZABETH'S MEDICAL CENTER
BERKSHIRE MEDICAL CENTER INC - 1
MARLBOROUGH HOSPITAL
SIGNATURE HEALTHCARE BROCKTON HOSPITAL
CLINTON HOSPITAL ASSOCIATION
BETH 

...

CATSKILL REGIONAL MEDICAL CENTER
ROCKEFELLER UNIVERSITY HOSPITAL
UNIVERSITY HOSPITAL ( STONY BROOK )
UNITED HEALTH SERVICES HOSPITALS, INC
ST JOHN'S EPISCOPAL HOSPITAL AT SOUTH SHORE
WOODHULL MEDICAL AND MENTAL HEALTH CENTER
INTERFAITH MEDICAL CENTER
ST BARNABAS HOSPITAL
ST CATHERINE OF SIENA HOSPITAL
MONROE COMMUNITY HOSPITAL
HELEN HAYES HOSPITAL
SUNNYVIEW HOSPITAL AND REHABILITATION CENTER
TRI-TOWN REGIONAL HEALTHCARE
BURDETT CARE CENTER
CUBA MEMORIAL HOSPITAL, INC
ELIZABETHTOWN COMMUNITY HOSPITAL
CATSKILL REGIONAL MEDICAL CENTER - G HERMANN SITE
MARGARETVILLE MEMORIAL HOSPITAL
O'CONNOR HOSPITAL
MOSES-LUDINGTON HOSPITAL
CLIFTON FINE HOSPITAL
RIVER HOSPITAL, INC
ELLENVILLE REGIONAL HOSPITAL
LITTLE FALLS HOSPITAL
DELAWARE VALLEY HOSPITAL, INC
SCHUYLER HOSPITAL, INC
SOLDIERS AND SAILORS MEMORIAL HOSPITAL OF YATES
GOUVERNEUR HOSPITAL
COMMUNITY MEMORIAL HOSPITAL, INC
LEWIS COUNTY GENERAL HOSPITAL
CARTHAGE AREA HOSPITAL, INC
MEDINA MEMORIAL HOSPITAL
BLYTHEDALE CHILDREN'S HOSPITAL
CAROLINAS

SANTIAM HOSPITAL
ADVENTIST MEDICAL CENTER
PROVIDENCE PORTLAND MEDICAL CENTER
WILLAMETTE VALLEY MEDICAL CENTER
PROVIDENCE MEDFORD MEDICAL CENTER
PROVIDENCE MILWAUKIE HOSPITAL
LEGACY MERIDIAN PARK MEDICAL CENTER
BAY AREA HOSPITAL
KAISER SUNNYSIDE MEDICAL CENTER
SACRED HEART MEDICAL CENTER - RIVERBEND
KAISER FOUNDATION HOSPITAL - WESTSIDE
PEACEHEALTH COTTAGE GROVE COMMUNITY MEDICAL CENTER
SAMARITAN NORTH LINCOLN HOSPITAL
PROVIDENCE SEASIDE HOSPITAL
SOUTHERN COOS HOSPITAL & HEALTH CENTER
BLUE MOUNTAIN HOSPITAL
WALLOWA MEMORIAL HOSPITAL
HARNEY DISTRICT HOSPITAL
SALEM HEALTH WEST VALLEY
LAKE DISTRICT HOSPITAL
PIONEER MEMORIAL HOSPITAL
LOWER UMPQUA HOSPITAL DISTRICT
COQUILLE VALLEY HOSPITAL DISTRICT
ST CHARLES PRINEVILLE
SAMARITAN PACIFIC COMMUNITY HOSPITAL
SAINT ALPHONSUS MEDICAL CENTER - BAKER CITY, INC
PEACE HARBOR MEDICAL CENTER
TILLAMOOK REGIONAL MEDICAL CENTER
PROVIDENCE HOOD RIVER MEMORIAL HOSPITAL
ST ANTHONY HOSPITAL
COLUMBIA MEMORIAL HOSPITAL
GRANDE RONDE HOSPITAL
CURRY GENERAL HOSPI

UNITED REGIONAL HEALTH CARE SYSTEM
ST JOSEPH REGIONAL HEALTH CENTER
PARKLAND HEALTH AND HOSPITAL SYSTEM
UNIVERSITY OF TEXAS MEDICAL BRANCH
BAYLOR UNIVERSITY MEDICAL CENTER
CITIZENS MEDICAL CENTER
UNIVERSITY MEDICAL CENTER OF EL PASO
VALLEY BAPTIST MEDICAL CENTER- BROWNSVILLE
LAREDO MEDICAL CENTER
CHRISTUS GOOD SHEPHERD MEDICAL CENTER MARSHALL
VHS HARLINGEN HOSPITAL COMPANY LLC
CHRISTUS SOUTHEAST TEXAS- ST ELIZABETH
ST JOSEPH MEDICAL CENTER
CHRISTUS GOOD SHEPHERD MEDICAL CENTER- LONGVIEW
JPS HEALTH NETWORK
COVENANT MEDICAL CENTER
PROVIDENCE HEALTH CENTER
UT SOUTHWESTERN UNIVERSITY HOSPITAL
CHRISTUS SPOHN HOSPITAL CORPUS CHRISTI
METHODIST DALLAS MEDICAL CENTER
SCOTT & WHITE MEDICAL CENTER - TEMPLE
ROLLING PLAINS MEMORIAL HOSPITAL
SETON MEDICAL CENTER AUSTIN
BAPTIST MEDICAL CENTER
TEXAS HEALTH ARLINGTON MEMORIAL HOSPITAL
MEMORIAL HERMANN TEXAS MEDICAL CENTER
BRAZOSPORT REGIONAL HEALTH SYSTEM
ANSON GENERAL HOSPITAL
BAYLOR MEDICAL CENTER AT IRVING
TITUS REGIONAL MEDICAL CENTER
CHRISTUS SPOH

PROVIDENCE CENTRALIA HOSPITAL
ST CLARE HOSPITAL
PROVIDENCE ST PETER HOSPITAL
SWEDISH MEDICAL CENTER / CHERRY HILL
SWEDISH EDMONDS HOSPITAL
SWEDISH MEDICAL CENTER
ST JOSEPH MEDICAL CENTER
GRAYS HARBOR COMMUNITY HOSPITAL
SAMARITAN HOSPITAL
YAKIMA VALLEY MEMORIAL HOSPITAL
TOPPENISH COMMUNITY HOSPITAL
HARRISON MEMORIAL HOSPITAL
PEACHEALTH ST JOHN MEDICAL CENTER
MULTICARE DEACONESS HOSPITAL
WALLA WALLA GENERAL HOSPITAL
PEACEHEALTH SOUTHWEST MEDICAL CENTER
OVERLAKE HOSPITAL MEDICAL CENTER
GROUP HEALTH CENTRAL HOSPITAL
KENNEWICK GENERAL HOSPITAL
PROVIDENCE SACRED HEART MEDICAL CENTER
KADLEC REGIONAL MEDICAL CENTER
CASCADE VALLEY HOSPITAL
HARBORVIEW MEDICAL CENTER
OLYMPIC MEDICAL CENTER
PROVIDENCE HOLY FAMILY HOSPITAL
MULTICARE GOOD SAMARITAN HOSPITAL
EVERGREEN HEALTH MONROE
VALLEY MEDICAL CENTER
ST JOSEPH MEDICAL CENTER
MULTICARE VALLEY HOSPITAL
EVERGREENHEALTH MEDICAL CENTER
TACOMA GENERAL ALLENMORE HOSPITAL
CAPITAL MEDICAL CENTER
ST FRANCIS COMMUNITY HOSPITAL
CONFLUENCE HEALTH- WENATCHEE VA

Querying https://api.yelp.com/v3/businesses/catholic-medical-center-manchester ...
catholic-medical-center-manchester 2.0 23 https://www.yelp.com/biz/catholic-medical-center-manchester 42.9916097 -71.4736955
